In [127]:
import pandas as pd 
import numpy as np
data = pd.read_csv('/Users/christianmatthewgarduno/Downloads/streaming_titles_clean.csv')
data = data[["title", "director","cast","score",]]

Reducing the columns in the data set. 

In [128]:
stats_data=data.sample(frac=.5,random_state=111)
norm_data = data.drop(stats_data.index)
director_scores = stats_data
cast_scores = stats_data.assign(cast=stats_data['cast'].str.split(',')).explode('cast')
cast_scores

,title,director,cast,score
5802,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan",Noah Kaye Bentley,27.0
5802,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan",Brennley Brown,27.0
5802,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan",Zach Callison,27.0
5802,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan",Nicolas Cantu,27.0
5802,Dragons: Rescue Riders: Secrets of the Songwing,"Greg Rankin, TJ Sullivan",Skai Jackson,27.0
...,...,...,...,...
5249,The Lost Pirate Kingdom,NaN,Mia Tomlinson,64.0
5249,The Lost Pirate Kingdom,NaN,Kevin Howarth,64.0
5249,The Lost Pirate Kingdom,NaN,Mark Gillis,64.0
5249,The Lost Pirate Kingdom,NaN,George Watkins,64.0


Here I split the main dataset into two, and from there set two seperate group bys one for directors and one for the cast memebers. For the cast members I had to split them because they were in a list. 

In [129]:
cast_scores['Number_MoviesShows'] = cast_scores['cast'].apply(lambda x: (cast_scores['cast'] == x).sum())
director_scores['Number_MoviesShows'] = director_scores['director'].apply(lambda x: (director_scores['director'] == x).sum())

This function is suppose to account the number of movies or shows a cast member has been in as well as for the director split. 

In [130]:
director_scores=director_scores.groupby(['director','Number_MoviesShows' ])['score'].mean()
director_scores = pd.DataFrame(director_scores).reset_index()
director_scores.columns = ['director', 'Number_MoviesShows_dir','dir_average_score']
cast_scores2=cast_scores.groupby(['cast','Number_MoviesShows'])['score'].mean()
cast_count = pd.DataFrame(cast_scores2).reset_index()
cast_count.columns = ['cast', 'Number_MoviesShows_cast','cast_average_score']
cast_count
director_scores

,director,Number_MoviesShows_dir,dir_average_score
0,20th_century_fox,1,58.0
1,A. L. Vijay,1,12.0
2,A.J. Murugan,1,28.0
3,A.R. Murugadoss,1,45.0
4,Aanand Rai,1,60.0
...,...,...,...
2900,Antonio Margheriti,1,3.0
2901,"Raoul Walsh, Mario Bava",1,17.0
2902,Álex de la Iglesia,1,65.0
2903,Álvaro Delgado-Aparicio L.,1,64.0


Converted from series due to group by back to dataframes. Cast_count is complete just needs to be merged with the other half of the parent dataset (don't have varaince at the moment). As well as Director Scores. 

In [139]:
dir_tot = pd.merge(norm_data, director_scores, on='director')
total_data = pd.merge(dir_tot,cast_count,on='cast')
total_data

,title,director,cast,score,Number_MoviesShows_dir,dir_average_score,Number_MoviesShows_cast,cast_average_score
0,John Mulaney: New in Town,Ryan Polito,John Mulaney,79.0,4,50.500000,1,72.000000
1,John Mulaney: The Comeback Kid,Rhys Thomas,John Mulaney,84.0,1,72.000000,1,72.000000
2,Jeff Dunham: Controlled Chaos,"Michael Simon, Matthew McNeil",Jeff Dunham,53.0,1,32.000000,4,41.000000
3,Jeff Dunham: Relative Disaster,Michael Simon,Jeff Dunham,37.0,1,36.000000,4,41.000000
4,Iliza Shlesinger: War Paint,Jay Chapman,Iliza Shlesinger,40.0,5,27.400000,1,64.000000
5,Iliza Shlesinger: Unveiled,Steve Paley,Iliza Shlesinger,46.0,1,64.000000,1,64.000000
6,Iliza Shlesinger: Confirmed Kills,Bobcat Goldthwait,Iliza Shlesinger,57.0,2,41.000000,1,64.000000
7,Jim Gaffigan: Obsessed,Jay Chapman,Jim Gaffigan,64.0,5,27.400000,4,52.750000
8,Jim Gaffigan: King Baby,Troy Miller,Jim Gaffigan,58.0,2,28.000000,4,52.750000
9,Jim Gaffigan: Cinco,Jeannie Gaffigan,Jim Gaffigan,61.0,1,56.000000,4,52.750000
